In [1]:
import subprocess
import time

import modifyCode

In [2]:
# input:
#   - code (string)
#   - tmpFilename (string)
#   - cFilename (string) (compiled file name)
#   - inputTestcases (list(bytes))
#   - outputTestcases (list(bytes))
#   - singleTestTimeout (int)
#   - stopFast (bool, optional) (if stopFast true, return immediately when the test fails)
# output:
#   - totalResultCode (0 for success. 1 for testcase error. 2 for compile error. 3 for no testcases)
#   - resultCode for each testcases (list(int)) (0 for success. 1 for W/A. 2 for timeout. 3 for other runtime errors.)
def codeTest(code, tmpFilename, cFilename, inputTestcases, outputTestcases, singleTestTimeout, stopFast=True):
    # exceptional case : no testcases
    if len(inputTestcases) < 1:
        return (3, [])
    
    # put code into file
    with open(tmpFilename, 'w') as file:
        file.write(code)
    # compile
    p = subprocess.run(['g++', tmpFilename, '-o', cFilename], shell=False)
    if p.returncode != 0:
        return (2, [])
    
    # run testcases
    totalResultCode = 0
    resultCodes = []
    for i, o in zip(inputTestcases, outputTestcases):
        try:
            p = subprocess.run([cFilename], input=i, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=singleTestTimeout)
        except subprocess.TimeoutExpired:
            totalResultCode = 1
            resultCodes.append(2)
            if stopFast:
                return totalResultCode, resultCodes
            else:
                continue
        except:
            totalResultCode = 1
            resultCodes.append(3)
            if stopFast:
                return totalResultCode, resultCodes
            else:
                continue
        if o == p.stdout:
            resultCodes.append(0)
        else:
            totalResultCode = 1
            resultCodes.append(1)
            if stopFast:
                return totalResultCode, resultCodes
    return totalResultCode, resultCodes

In [7]:
# input:
#   - code (list contains the whole code line by line.) (list(string))
#   - vscm (list(dictionary))
#   - tmpFilename (string)
#   - cFilename (string)
#   - modNum (int) (the number of modifications)
#   - inputTestcases (list(bytes))
#   - outputTestcases (list(bytes))
#   - singleTestTimeout (represent timeout seconds.)
#   - totalTimeout (represent timeout seconds. int, optional)  (non-positive value for no-limit)
# output: multiple return values.
#   - modified code String (string) (If it fails to find, return '')
#   - modified codeline Indices, start from 0 (list(int))
#   - selected vscmIndices for successful modification (list(int))
#   - returnStatus(int) (0 for success, 1 for not-found, 2 for timeout, 3 for no testcases)
#   - resultCodes (list(int)) (results for each testcases: following the return value of codeTest.)
#   - elapsed time (int) (seconds)
from itertools import combinations

def findVarErr2(code, vscm, tmpFilename, cFilename, modNum, inputTestcases, outputTestcases, singleTestTimeout, totalTimeout=0):
    if len(inputTestcases) < 1:
        return ('', [], 3, [], 0)
    
    startTime = time.time()
    timeout = startTime + totalTimeout
    validModificationCount = 0
    for mN in range(1, modNum + 1):
        # mT for modification-tuple
        # mL for modification-list
        for mT in combinations(range(len(vscm)), mN):
            # timeout check
            if totalTimeout > 0 and time.time() > timeout:
                return ('', [], [], 2, [], (time.time() - startTime))
        
            # modify Code
            mc, mcis, rs = modifyCode.modCodesFull(code, vscm, mT)
            
            if rs != 0:
                continue
                
            #  test code witih testcases
            trc, rcs = codeTest(mc, tmpFilename, cFilename, inputTestcases, outputTestcases, singleTestTimeout)
            
            # debugging - count the number of iteration
            validModificationCount += 1
            print('===========================================')
            print('validModificationCount   # ' + str(validModificationCount))
            print('TEST RESULTS:')
            print(rcs)
            print('MODIFIED SRC:')
            print(mc)
            print('')
            
            if trc == 0:
                return (mc, mcis, list(mT), 0, rcs, (time.time() - startTime))
    return ('', [], [], 1, [], (time.time() - startTime))
        

## Naive Solver

In [8]:
### NAIVE SOLUTION
# expected directory layout
# testDir
# - codefname
# - input
#   - 1.txt
#   - 2.txt
#   - ...
# - output
#   - 1.txt
#   - 2.txt
#   - ...

import os
from modifyCode import readVscm, modCodeFull
from makeVscm import genVscm

def naiveSolution2(testDir, codefname, modNumLimit=2, singleTestTimeout=2, testcaseLimit=1000):
    # set filenames
    codeFilename = os.path.join(testDir, codefname)
    vscmFilename = os.path.join(testDir, 'vscm.csv')
    tmpFilename = os.path.join(testDir, 'tmpFile.cpp')
    cFilename = os.path.join(testDir, 'cfile.out')
    inputTCDir = os.path.join(testDir, 'input')
    outputTCDir = os.path.join(testDir, 'output')
    
    # make code string list
    with open(codeFilename, 'r') as codefile:
        code = codefile.readlines()
    
    # make and read vscm
    genVscm(codeFilename, vscmFilename)
    vscm = readVscm(vscmFilename)
    
    # read testcases
    inputTestcases = []
    outputTestcases = []
    for i in range(1, testcaseLimit):
        fn = str(i)
        try:
            with open(os.path.join(inputTCDir, fn), 'rb') as itcf:
                with open(os.path.join(outputTCDir, fn), 'rb') as otcf:
                    inputTestcases.append(itcf.read())
                    outputTestcases.append(otcf.read())
        except IOError:
            break
    
    # run!
    src, linNums, vscmIndices, rs, rcs, etime = findVarErr2(code, vscm, tmpFilename, cFilename, modNumLimit, inputTestcases, outputTestcases, singleTestTimeout)
    print('')
    print('====================================== RESULT')
    print('====================================== RESULT')
    print('====================================== RESULT')
    print('====================================== SRC')
    print(src)
    print('====================================== RS')
    print(rs)
    print('====================================== Result code for each testcases')
    print(rcs)
    print('====================================== VSCM Indices')
    print(vscmIndices)
    print('====================================== modifiedLineNum')
    print(linNums)
    print('====================================== ELAPSED TIME')
    print(etime)
    print('')
    return rs
    

## TEST

In [9]:
TEST_FLAG=True

In [10]:
# run naiveSolution
if TEST_FLAG:
    testDir = '/home/ubuntu/workspace/pyws/modifyCpp/naiveSolverTestDir/test1'
    codefname = 'code.cpp'

    naiveSolution2(testDir, codefname)

validModificationCount   # 1
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> z >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 2
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> w >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 3
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> y >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 4
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> x >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 5
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> 

validModificationCount   # 34
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> z >> w >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 35
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> z >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 36
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> z >> x >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 37
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> z >> y >> z;
    cout << z+x << endl;
    return 0;
}

validModificationCount   # 38
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> 

validModificationCount   # 67
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> y >> y >> z;
    cout << w+y << endl;
    return 0;
}

validModificationCount   # 68
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> y >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 69
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> z >> y >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 70
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> y >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 71
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> 

validModificationCount   # 100
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> x >> y >> z;
    cout << x+x << endl;
    return 0;
}

validModificationCount   # 101
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> x >> y >> z;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 102
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> x >> y >> w;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 103
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    cin >> w >> x >> y >> y;
    cout << w+x << endl;
    return 0;
}

validModificationCount   # 104
TEST RESULTS:
[1]
MODIFIED SRC:
#include <iostream>

using namespace std;

int main(void){
    int w, x, y, z;
    ci